<a href="https://colab.research.google.com/github/jandeilsonxavier/Recomendacao-Produtos-Loja-Virtual/blob/main/vendas_bm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Carregar o dataset
file_path = 'vendas_bm.csv'

df = pd.read_csv(file_path, sep=";")
# Mostrar todas as colunas

df.columns

Index(['Order Date', 'Order ID', 'Product', 'Product_ean', 'catégorie',
       'Purchase Address', 'Quantity Ordered', 'Price Each', 'Cost price',
       'turnover', 'margin'],
      dtype='object')

In [ ]:
# # Passo 1: Remover os pontos como separadores de milhares
# df['Total Pedido'] = df['Total Pedido'].apply(lambda x: x.replace('.', '') if '.' in x else x)

# # # Passo 2: Substituir vírgulas por pontos para separar casas d
# df['Total Pedido'] = df['Total Pedido'].str.replace(',', '.', regex=True)

# # # Passo 3: Converter para float
# df['Total Pedido'] = df['Total Pedido'].astype('float64')
# df['Total Pedido']

,Total Pedido
0,24.77
1,24.77
2,252.98
3,252.98
4,252.98
...,...
3160,43.86
3161,43.86
3162,43.86
3163,14.16


In [ ]:
df['Total Pedido'].describe()

,Total Pedido
count,3165.00000
mean,186.60042
std,433.94600
min,0.86000
25%,35.70000
50%,71.10000
75%,153.10000
max,3600.00000


In [ ]:
df.shape

(3165, 41)

In [ ]:
# Agrupar os produtos por pedido em uma única lista
basket = df.groupby('Número pedido')['Produto'].apply(list).reset_index()

# Convert all items in the 'Produto' column to strings before fitting
basket['Produto'] = basket['Produto'].apply(lambda x: [str(item) for item in x])
print(basket.head())
basket.shape

   Número pedido                                            Produto
0            692  [Protoboard 400 Pontos, Fonte Ajustável para P...
1            693  [Arduino Uno R3 SMD ATmega328P Tipo:Com Cabo, ...
2            694   [Kit Arduino Uno R3 Iniciante 78 Peças + Brinde]
3            695  [Driver Motor de Passo DRV8825 + Dissipador de...
4            696  [Válvula Solenoide para Água (1/2 x 1/2) 220V ...


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1041, 2)

Converter para Formato One-Hot Encoding  

1.   Item da lista
2.   Item da lista


Para o Apriori, precisamos converter a lista de produtos em uma matriz binária (onde cada coluna representa um produto).

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

# Aplicar o TransactionEncoder
te = TransactionEncoder()
te_data = te.fit(basket['Produto']).transform(basket['Produto'])
df_encoded = pd.DataFrame(te_data, columns=te.columns_)

print(df_encoded.head())
df_encoded.shape

**Aplicar o Algoritmo Apriori**  
Usaremos o algoritmo Apriori para encontrar combinações frequentes de produtos.

In [ ]:
# Importar bibliotecas necessárias
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Gerar conjuntos de itens frequentes
frequent_itemsets = apriori(df_encoded, min_support=0.002, use_colnames=True)

# Gerar regras de associação
rules = association_rules(frequent_itemsets, metric='confidence',min_threshold=0.005, num_itemsets=1)

# Exibir resultados
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head())
rules.shape


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                                         antecedents  \
0  (Acelerômetro e Giroscópio 3 Eixos GY-521 MPU-...   
1                          (LDR - Sensor De Luz 5mm)   
2  (Acelerômetro e Giroscópio 3 Eixos GY-521 MPU-...   
3           (Módulo ESP32 Wifi e Bluetooth 30 Pinos)   
4  (Acelerômetro e Giroscópio 3 Eixos GY-521 MPU-...   

                                         consequents   support  confidence  \
0                          (LDR - Sensor De Luz 5mm)  0.002882    0.150000   
1  (Acelerômetro e Giroscópio 3 Eixos GY-521 MPU-...  0.002882    0.115385   
2           (Módulo ESP32 Wifi e Bluetooth 30 Pinos)  0.002882    0.150000   
3  (Acelerômetro e Giroscópio 3 Eixos GY-521 MPU-...  0.002882    0.047619   
4  (Regulador de Tensão LM2596 Conversor DC-DC St...  0.003842    0.200000   

       lift  
0  6.005769  
1  6.005769  
2  2.478571  
3  2.478571  
4  6.940000  


(27216, 14)

In [ ]:
# Transformar conjuntos em strings para exibição mais clara
# rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
# rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x)[0])
rules['consequents'] = rules['consequents'].apply(lambda x: list(x)[0])

# Exibir as 5 primeiras regras
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head())

                                         antecedents  \
0  Acelerômetro e Giroscópio 3 Eixos GY-521 MPU-6050   
1                            LDR - Sensor De Luz 5mm   
2  Acelerômetro e Giroscópio 3 Eixos GY-521 MPU-6050   
3             Módulo ESP32 Wifi e Bluetooth 30 Pinos   
4  Acelerômetro e Giroscópio 3 Eixos GY-521 MPU-6050   

                                         consequents   support  confidence  \
0                            LDR - Sensor De Luz 5mm  0.002882    0.150000   
1  Acelerômetro e Giroscópio 3 Eixos GY-521 MPU-6050  0.002882    0.115385   
2             Módulo ESP32 Wifi e Bluetooth 30 Pinos  0.002882    0.150000   
3  Acelerômetro e Giroscópio 3 Eixos GY-521 MPU-6050  0.002882    0.047619   
4  Regulador de Tensão LM2596 Conversor DC-DC Ste...  0.003842    0.200000   

       lift  
0  6.005769  
1  6.005769  
2  2.478571  
3  2.478571  
4  6.940000  


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Aplicação no Sistema "Compre Junto"

In [ ]:
# # Filtrar regras de interesse
# # rules_filtered = rules[rules['confidence'] > 0.5]  # Apenas regras com alta confiança

# # Transformar 'antecedents' e 'consequents' em strings para manipulação
# rules_filtered['antecedents'] = rules['antecedents'].apply(lambda x: list(x)[0])
# rules_filtered['consequents'] = rules['consequents'].apply(lambda x: list(x)[0])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-23-2c89c459fa1a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rules_filtered['antecedents'] = rules_filtered['antecedents'].apply(lambda x: list(x)[0])
<ipython-input-23-2c89c459fa1a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://

In [ ]:
# Produto que você quer analisar
produto_analise = 'Arduino Uno R3 SMD ATmega328P'

# Filtrar as regras de associação onde o produto está como antecedente
regras_produto = rules[rules['antecedents'] == produto_analise]

# # Transformar 'antecedents' e 'consequents' em strings para manipulação
# regras_produto['antecedents'] = rules['antecedents'].apply(lambda x: list(x)[0])
# regras_produto['consequents'] = rules['consequents'].apply(lambda x: list(x)[0])

# Exibir as regras relacionadas ao produto
print(regras_produto[['antecedents', 'consequents','support', 'confidence', 'lift']])

In [ ]:
# Ordenar as regras por suporte em ordem decrescente e pegar os 5 primeiros produtos
top_products = regras_produto.drop_duplicates(subset='consequents')
top5_products = top_products.sort_values(by='support', ascending=False).head(5)
# Exibir os 5 primeiros produtos com maior suporte
print(top5_products[['consequents','support', 'confidence', 'lift']])

                                         consequents   support  confidence  \
32          Cabo Jumper 20cm Macho-Macho 40 Unidades  0.005764    0.272727   
34                        Driver Motor Ponte H L298N  0.005764    0.272727   
30          Cabo Jumper 20cm Macho-Fêmea 40 Unidades  0.003842    0.181818   
39  Fonte de Alimentação Chaveada DC 12V 5A Metálica  0.003842    0.181818   
40      Módulo Sensor Distância Ultrassônico HC-SR04  0.003842    0.181818   

         lift  
32   5.678182  
34  12.904959  
30   3.052786  
39  15.772727  
40   4.980861  


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# prompt: salvar em um arquivo csv
regras_produto = regras_produto[['antecedents', 'consequents','support', 'confidence', 'lift']]
# Salvar as regras de associação em um arquivo excel
regras_produto.to_excel('regras_associacao_esp1.xlsx', index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Importar bibliotecas necessárias
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder

# Carregar o dataset
file_path = 'vendas_bm.csv'

df = pd.read_csv(file_path, sep=";")

# Agrupar os produtos por pedido em uma única lista
basket = df.groupby('Número pedido')['Produto'].apply(list).reset_index()

# Convert all items in the 'Produto' column to strings before fitting
basket['Produto'] = basket['Produto'].apply(lambda x: [str(item) for item in x])

# Converter para Formato One-Hot Encoding
te = TransactionEncoder()
te_data = te.fit(basket['Produto']).transform(basket['Produto'])
df_encoded = pd.DataFrame(te_data, columns=te.columns_)

# Gerar conjuntos de itens frequentes
frequent_itemsets = apriori(df_encoded, min_support=0.002, use_colnames=True)

# Gerar regras de associação
rules = association_rules(frequent_itemsets, metric='confidence',min_threshold=0.005, num_itemsets=1)

# Transformar conjuntos em strings para exibição mais clara
rules['antecedents'] = rules['antecedents'].apply(lambda x: list(x)[0])
rules['consequents'] = rules['consequents'].apply(lambda x: list(x)[0])

produto_para_analise = input("Insira o produto para análise: ")

if produto_para_analise not in rules['antecedents'].unique():
    print(f"O produto '{produto_para_analise}' não foi encontrado.")
    print("Recomendações não disponíveis.")
else:
    # Filtrar as regras de associação onde o produto está como antecedente
    regras_produto = rules[rules['antecedents'] == produto_para_analise]

    top_products = regras_produto.drop_duplicates(subset='consequents')

    # Ordenar as regras por suporte em ordem decrescente e pegar os 5 primeiros produtos
    top5_products = top_products.sort_values(by='support', ascending=False).head(10)

    # Exibir os 5 primeiros produtos com maior suporte
    print(top5_products[['consequents','support', 'confidence']])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Insira o produto para análise: Protoboard 830 Pontos
                                            consequents   support  confidence
1012  Resistores Vários Modelos 1/4w 10 Unidades Res...  0.010567        0.22
247            Cabo Jumper 20cm Macho-Fêmea 40 Unidades  0.010567        0.22
321            Cabo Jumper 20cm Macho-Macho 40 Unidades  0.010567        0.22
655                Kit Led Difuso 5mm 4 Cores 40 Peças.  0.007685        0.16
1005  Resistores Vários Modelos 1/4w 10 Unidades Res...  0.007685        0.16
570                     Fonte Ajustável para Protoboard  0.006724        0.14
1006  Resistores Vários Modelos 1/4w 10 Unidades Res...  0.006724        0.14
981                            Potenciômetro Linear 10k  0.006724        0.14
452         Conector de Bateria 9V Tipo:Com conector P4  0.005764        0.12
924        Módulo Sensor Distância Ultrassônico HC-SR04  0.005764        0.12
